In [1]:
import os
# Find the latest version of spark 3.2 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.3'
spark_version = 'spark-3.2.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://archive.ubuntu.com/ubuntu focal InRelease
Get:2 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  Release
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease [18.1 kB]
Get:8 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:9 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Get:14 http://archive.ubuntu.com/ubuntu foc

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M17-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [3]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Books_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Books_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   25933450| RJOVP071AVAJO|0439873800|      84656342|There Was an Old ...|           Books|          5|            0|          0|   N|                Y|          Five Stars|I love it and so ...| 2015-08-31|
|         US|    1801372|R1ORGBETCDW3AI|1623953553|     729938122|      I Saw a Friend|           Books|          5|    

In [4]:
# Create the vine_table DataFrame
vine_df = df.select(["review_id","star_rating","helpful_votes","total_votes","vine","verified_purchase"])

In [5]:
# Convert star_rating from sring to integer
from pyspark.sql.types import IntegerType
vine_df = vine_df.withColumn("star_rating", vine_df["star_rating"].cast(IntegerType()))

In [6]:
# Show data types
vine_df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)



In [7]:
# Show data frame
vine_df.show(10)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RJOVP071AVAJO|          5|            0|          0|   N|                Y|
|R1ORGBETCDW3AI|          5|            0|          0|   N|                Y|
| R7TNRFQAOUTX5|          5|            0|          0|   N|                Y|
|R2GANXKDIFZ6OI|          5|            0|          0|   N|                N|
|R2NYB6C3R8LVN6|          5|            2|          2|   N|                Y|
|R13U5PBJI1H94K|          2|            1|          1|   N|                N|
|R1H8UVH990F8VE|          5|            2|          2|   N|                N|
|R2MC0N30WZMRQ5|          5|            0|          0|   N|                Y|
| R2NO2HXK16Y4J|          5|            0|          0|   N|                Y|
|R245YIAVJK82ZL|          5|            0|          0|   N|     

In [10]:
# Step 1: Create a new DataFrame that filters the data to retrieve all the rows where the total_votes count is greater than or equal to 20
vine_total_votes_df = vine_df.filter("total_votes>=20")
vine_total_votes_df.show(10)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3T6LR4H11XRAN|          5|           26|         26|   N|                Y|
|R3QFNFMFWOQIBO|          3|           15|         25|   N|                Y|
|R24S2SV0ZIFIMV|          1|           41|         78|   N|                N|
|R2QAPWF1M5863L|          1|            5|         23|   N|                N|
|R3OCONKYE1A047|          5|           33|         38|   N|                N|
|R28WZIA84DUP6C|          2|            4|         43|   N|                N|
|R2U7YNJZ5JHWVD|          5|           23|         25|   N|                Y|
|R1XVYF74WCJKO3|          1|           47|         49|   N|                N|
|R3KKWNG5VABDYC|          5|           31|         32|   Y|                N|
|R3KK9MHFLY2BVW|          5|           21|         22|   N|     

In [13]:
# Step 2: Create a new DataFrame that filters the data to retrieve all the rows where 
# the number of helpful_votes divided by total_votes is equal to or greater than 50%

# Add a new column calculating helpful_votes / total_votes
vine_helpful_votes_df = vine_total_votes_df.withColumn("Votes_Percent", df["helpful_votes"] / df["total_votes"])
vine_helpful_votes_df.show()




+--------------+-----------+-------------+-----------+----+-----------------+--------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|       Votes_Percent|
+--------------+-----------+-------------+-----------+----+-----------------+--------------------+
|R3T6LR4H11XRAN|          5|           26|         26|   N|                Y|                 1.0|
|R3QFNFMFWOQIBO|          3|           15|         25|   N|                Y|                 0.6|
|R24S2SV0ZIFIMV|          1|           41|         78|   N|                N|  0.5256410256410257|
|R2QAPWF1M5863L|          1|            5|         23|   N|                N| 0.21739130434782608|
|R3OCONKYE1A047|          5|           33|         38|   N|                N|   0.868421052631579|
|R28WZIA84DUP6C|          2|            4|         43|   N|                N| 0.09302325581395349|
|R2U7YNJZ5JHWVD|          5|           23|         25|   N|                Y|                0.92|
|R1XVYF74W

In [15]:
# Filter the data frame to only include rows where the new column's value is .5 or above

vine_helpful_votes_df = vine_helpful_votes_df.filter("Votes_Percent>=0.5")
vine_helpful_votes_df.show(10)


+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|     Votes_Percent|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R3T6LR4H11XRAN|          5|           26|         26|   N|                Y|               1.0|
|R3QFNFMFWOQIBO|          3|           15|         25|   N|                Y|               0.6|
|R24S2SV0ZIFIMV|          1|           41|         78|   N|                N|0.5256410256410257|
|R3OCONKYE1A047|          5|           33|         38|   N|                N| 0.868421052631579|
|R2U7YNJZ5JHWVD|          5|           23|         25|   N|                Y|              0.92|
|R1XVYF74WCJKO3|          1|           47|         49|   N|                N|0.9591836734693877|
|R3KKWNG5VABDYC|          5|           31|         32|   Y|                N|           0.96875|
|R3KK9MHFLY2BVW|          5|  

In [16]:
# Step 3: Create a new DataFrame that filters the data to retrieve all the rows where a review was written as part of the Vine program (paid), vine == 'Y'.

vine_paid_df = vine_helpful_votes_df.filter(vine_helpful_votes_df["vine"]=="Y")
vine_paid_df.show(10)

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|     Votes_Percent|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R3KKWNG5VABDYC|          5|           31|         32|   Y|                N|           0.96875|
|R397PIJYYVJ7PD|          4|           31|         32|   Y|                N|           0.96875|
| RP6Y2CTIFUYD0|          2|           31|         32|   Y|                N|           0.96875|
|R1FYRNM5U0S2VZ|          3|           20|         24|   Y|                N|0.8333333333333334|
| R1ME19UY4UNAO|          5|           53|         57|   Y|                N|0.9298245614035088|
|R1JW7ZHT2VLT1E|          3|           40|         45|   Y|                N|0.8888888888888888|
| R98UYXY43TURV|          4|           46|         50|   Y|                N|              0.92|
|R34VXM3EJDS78Z|          3|  

In [17]:
# Step 4: Repeat Step 3 to retrieve all the rows where a review was NOT written as part of the Vine program (upaid), vine == 'N'

vine_unpaid_df = vine_helpful_votes_df.filter(vine_helpful_votes_df["vine"]=="N")
vine_unpaid_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|     Votes_Percent|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R3T6LR4H11XRAN|          5|           26|         26|   N|                Y|               1.0|
|R3QFNFMFWOQIBO|          3|           15|         25|   N|                Y|               0.6|
|R24S2SV0ZIFIMV|          1|           41|         78|   N|                N|0.5256410256410257|
|R3OCONKYE1A047|          5|           33|         38|   N|                N| 0.868421052631579|
|R2U7YNJZ5JHWVD|          5|           23|         25|   N|                Y|              0.92|
|R1XVYF74WCJKO3|          1|           47|         49|   N|                N|0.9591836734693877|
|R3KK9MHFLY2BVW|          5|           21|         22|   N|                N|0.9545454545454546|
|R2OEP8MGX5K370|          5|  

In [ ]:
# Step 5: For the two types of reviews (unpaid vs paid), determine the: 
#Total number of reviews
#Number of 5-star reviews
#Percentage of 5-star reviews

In [24]:
#Calculate the total number of paid reviews
vine_paid_df.count()

5012

In [25]:
#Calculate the total number of unpaid reviews
vine_unpaid_df.count()

109297

In [21]:
#Calculate the number of 5-star paid reviews
vine_paid_df.filter("star_rating==5").count()

2031

In [22]:
#Calculate the number of 5-star upaid reviews
vine_unpaid_df.filter("star_rating==5").count()

49967

In [32]:
#Calculate the percentage of paid reviews that were 5 stars
percent_paid = 2031/5012*100
percent_paid

40.52274541101357

In [33]:
#Calculate the percentage of unpaid reviews that were 5 stars
percent_unpaid = 49967/109297*100
percent_unpaid

45.716716835777746